In [32]:
import mbuild as mb
from mbuild.lib.recipes import TiledCompound
import parmed as pmd
import numpy as np
import openmm
import warnings
from random import sample
from copy import deepcopy
import openbabel as ob
from openbabel import pybel
from copy import deepcopy
# from slab_builder import *
warnings.simplefilter("ignore")

In [2]:
import ase
from ase import Atoms
from ase.build import surface
from ase.visualize import view
from ase.io import write

In [3]:
from ase.build import fcc111
slab = fcc111('Al', size=(2,2,3), vacuum=10.0)
view(slab)

<Popen: returncode: None args: ['/home/robert/miniconda3/envs/dev/bin/python...>

In [4]:
from ase.build import fcc110
slab = fcc110('Al', size=(2,2,3), vacuum=10.0)
view(slab)

<Popen: returncode: None args: ['/home/robert/miniconda3/envs/dev/bin/python...>

In [16]:
a = 5.39
Pt3Rh = Atoms('ZnS',
              scaled_positions=[(0, 0, 0),
                                (0.25, 0.75, 0.75)],
              cell=[a, a, a],
              pbc=True)
s3 = surface(Pt3Rh, (1, 1, 0), 6)
s3.center(vacuum=20, axis=2)

Pt3Rh.set_chemical_symbols('SZn')
s4 = surface(Pt3Rh, (1, 1, 0), 9)
s4.center(vacuum=20, axis=2)

In [17]:
big_110 = ase.build.make_supercell(s3,([6,0,0],[0,6,0],[0,0,1]))

view(big_110)

<Popen: returncode: None args: ['/home/robert/miniconda3/envs/dev/bin/python...>

In [18]:
write("ZnS_slab_110_6x6x6.vacuum.xyz", big_110)

In [19]:
write("ZnS_slab_110_6x6x6.vacuum.lammps", big_110, "lammps-data")

In [20]:
from scipy.spatial.distance import pdist, squareform
import random


def select_coordinates(ports, num_to_select, min_distance):
    
    coords = [port.pos for port in ports]

    num_coords = len(coords)
    if num_to_select > num_coords:
        print("Warning: Not enough coordinates available to select.")
        return None
    
    distance_matrix = squareform(pdist(coords))

    valid_indices = set(range(num_coords))
    selected_ports = []

    for _ in range(num_to_select):
        if not valid_indices:
            print("Warning: Unable to find enough coordinates with the given constraints.")
            break

        index = random.choice(list(valid_indices))
        selected_ports.append(ports[index])
        valid_indices.remove(index)

        too_close_indices = set(np.where(distance_matrix[index] < min_distance)[0])
        valid_indices -= too_close_indices

    return selected_ports

In [21]:
# Attempt to place a ligand on the slab

def place_ligand(ligand_file, port, slab, slab_coords):
    
    # 1) Generate the ligand structure
    ligand = mb.load(ligand_file)

    # 2) Translate the ligand to the slab coordinate
    ligand.add(mb.Port(anchor=ligand[0], separation=0.12, orientation=(ligand[2].pos - ligand[5].pos)))
    
    bond_orientation = port.pos - port.anchor.pos
    bond_orientation = bond_orientation / np.linalg.norm(bond_orientation)
    
    mb.force_overlap(move_this=ligand,
                     from_positions=ligand['Port[0]'],
                     to_positions=port)
    
    slab.add(ligand, label="OLC[$]")
    
    # print(bond_orientation, orient)

    coords = np.array(slab_coords)  # Convert list to numpy array for easier manipulation
    # print(slab_coords)
    min_coords = np.min(slab_coords, axis=0)
    max_coords = np.max(slab_coords, axis=0)
    # dimensions = max_coords - min_coords

    # 3) Check if ligand's x,y coordinates are within bounds of slab
    for attempt in range(36):  # Rotate up to 360 degrees
        x_coords = ligand.xyz[:, 0]
        y_coords = ligand.xyz[:, 1]

        min_x, max_x = np.min(x_coords), np.max(x_coords)
        min_y, max_y = np.min(y_coords), np.max(y_coords)
        
        if (min_x >= min_coords[0] and max_x <= max_coords[0] and
            min_y >= min_coords[1] and max_y <= max_coords[1]):
            # 3a) If they are within the bounds, return True
            
            
            return True, ligand
        else:
            # 3b) If they are not, try rotating by 10 degrees
            ligand.translate(-port.pos)
            ligand.rotate(theta=np.deg2rad(10), around=bond_orientation)  # Rotate around z-axis
            ligand.translate(port.pos)
            # slab.remove(ligand)

    # If no suitable orientation found after rotating through all 360 degrees, return False
    slab.remove(ligand)
    return False, ligand

In [50]:
oleic_ion = mb.load("slab_oleic/test_oleic_ion.mol2")
test_slab = mb.load("ZnS_slab_110_6x6x6.xyz")

In [52]:
for atom in test_slab:
    if atom.name == "Zn":
        atom.charge = 2
    elif atom.name == "S":
        atom.charge = -2
    else:
        print("Unrecognized atom name!!")

In [53]:
pybel_slab = test_slab.to_pybel()
ff = pybel._forcefields["uff"]
success = ff.Setup(pybel_slab.OBMol)
if success:
    ff.GetCoordinates(pybel_slab.OBMol)
    outBGF = pybel.Outputfile('bgf', 'test_slab.bgf', overwrite=True)
    outBGF.write(pybel_slab)
    outBGF.close()
else:
    print("Error setting up forcefield")

## need to add charges to the slab:

`cat test_slab.bgf | sed 's/Zn     1 0  0.00000/Zn     1 0  2.00000/' > temp.bgf`

`cat temp.bgf | sed 's/S      1 0  0.00000/S     1 0 -2.00000/' > ase-slab.charged.bgf`

In [35]:
ff.Setup(pybel_slab.OBMol)

False

In [23]:
for Zn in test_slab:
    if Zn.element.symbol == 'Zn' and Zn.pos[2] > 1.9:
        port = mb.Port(anchor=Zn, orientation=[0,0,1], separation=0.05)
        test_slab.add(port)
        # print(Zn.pos)

In [28]:
4.5 * 3.2

14.4

In [26]:
ports = test_slab.all_ports()

coords = [port.pos for port in test_slab.all_ports()]

# Randomly select N sites from `coords`, with a minimum of X distance between them
# select_coordinates(coords, N, X)
# will throw a warning if unable to find specified N sites given coords + X criteria
selected_sites = select_coordinates(test_slab.all_ports(), 24, 0.1)

coord = ports[4]  # replace with your actual coordinate
slab_coords = [port.pos for port in test_slab.all_ports()]  # replace with your actual slab dimensions

ligands = []
missing_index = []

# using all possible ports here, even this is only giving me 16 slots. dont think the rotation function is working
for i,site in enumerate(test_slab.all_ports()):
    result, ligand = place_ligand("slab_oleic/test_oleic_ion.mol2", site, test_slab, slab_coords)
    if result == False:
        print(f"missed this one: {i}")
        missing_index.append(i)
    else:
        ligands.append(ligand)

missed this one: 0
missed this one: 1
missed this one: 2
missed this one: 10
missed this one: 11
missed this one: 12
missed this one: 13
missed this one: 14
missed this one: 15
missed this one: 16
missed this one: 17
missed this one: 18
missed this one: 19
missed this one: 20
missed this one: 21
missed this one: 22
missed this one: 23
missed this one: 24
missed this one: 25
missed this one: 26


In [27]:
test_slab.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [29]:
zn_bond_list = []
for bond in test_slab.bonds():
    if bond[0].name == 'Zn' or bond[1].name == 'Zn':
        zn_bond_list.append(bond)
for bond in zn_bond_list:
    test_slab.remove_bond(bond)

In [30]:
test_slab.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [46]:
test_oleic_clone = mb.load("slab_oleic/test_oleic_ion.mol2")

In [47]:
oleic_charges = []
with open("slab_oleic/oleic_charges.txt") as f:
    for line in f:
        element = line.strip().split()[0]
        charge = float(line.strip().split()[1])
        oleic_charges.append((element, charge))

In [48]:
for i,at in enumerate(test_oleic_clone):
    if at.element.symbol in oleic_charges[i][0]:
        at.charge = oleic_charges[i][1]

In [49]:
for i,ligand in enumerate(ligands):
    for j,particle in enumerate(ligand.particles()):
        test_oleic_clone[j].translate_to(particle.pos)
    test_oleic_clone.save(f"ligand_files/ligand_{i}.mol2", overwrite=True)